In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("Ingesta Bronze") \
    .master("spark://localhost:7077")\
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.sources.partitionOverwriteMode", "dynamic")\
    .config("hive.exe.dynamic.partition.mode", "nonstrict") \
    .getOrCreate()

from delta.tables import *

24/07/03 22:58:53 WARN Utils: Your hostname, besgam-Aspire-E1-471 resolves to a loopback address: 127.0.1.1; using 192.168.0.16 instead (on interface wlp9s0)
24/07/03 22:58:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/besgam/.ivy2/cache
The jars for the packages stored in: /home/besgam/.ivy2/jars
:: loading settings :: url = jar:file:/home/besgam/anaconda3/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6ff60dba-11ac-4bb1-9b84-df763114fa49;1.0
	confs: [default]
	found io.delta#delta-core_2.12;0.8.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glass

In [2]:
anoMes = 202312

In [3]:
def qualifyTypeColumn(df, columns, expressions, withColumn):
    for column, expression in zip(columns, expressions):
        if withColumn:
            df = df.withColumn(column, expression)
        else:
            df = df.withColumnRenamed(column, expression)
    return df

In [ ]:
consumidor = spark.read.parquet("hdfs://namenode:9000/data/consumidor/landing/*.parquet")

In [21]:
columns = ['dataAbertura', 'dataResposta', 'dataAnalise', 'dataRecusa', 'prazoResposta', 'procurouEmpresa', 'respondida']

In [23]:
expressions = [ from_unixtime(unix_timestamp(col('dataAbertura'), 'dd/MM/yyyy')).cast(DateType())
               , from_unixtime(unix_timestamp(col('dataResposta'), 'dd/MM/yyyy')).cast(DateType()) 
               , from_unixtime(unix_timestamp(col('dataAnalise'), 'dd/MM/yyyy')).cast(DateType()) 
               , from_unixtime(unix_timestamp(col('dataRecusa'), 'dd/MM/yyyy')).cast(DateType())               
               , from_unixtime(unix_timestamp(col('prazoResposta'), 'dd/MM/yyyy')).cast(DateType())
               , when(col('procurouEmpresa') == 'S', lit(1)).otherwise(lit(0)).cast(BooleanType())
               , when(col('respondida') == 'S', lit(1)).otherwise(lit(0)).cast(BooleanType())
              ]

In [26]:
consumidor = qualifyTypeColumn(consumidor, columns, expressions, True)

In [29]:
consumidor.write.partitionBy('datRefCarga').format("delta").mode("append").save("hdfs://namenode:9000/data/consumidor/bronze")